In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import statistics as stat
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import copy
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import Dataset

from sklearn.model_selection import train_test_split
import torch.optim as optim
import polars as pl


evaluate = False
# model_path = "/kaggle/input/custom_trans_random_epoch9/pytorch/default/1/torchcustom_random_epoch9.pth"

In [3]:
# !pip install scikit-learn



In [4]:
jane_street_real_time_market_data_forecasting_path = '/home/siddharth/jane_street_challenge_2024/data'

## LSTM Architecture
One of the major problem of LSTM is the overfitting, thus to try to prevent this a little we will add some gaussian noise to the model. This helps a little but not perfectly.
### Gaussian Noise
We will add noise during the training part only, this is because we dont want it to affect the test results, only prevent the model from overfitting on the training data.

In [5]:
class GaussianNoise(nn.Module):
    def __init__(self, std=0.1):
        super().__init__()
        self.std = std
        self.training = False
    def forward(self, x):
        if self.training: 
            noise = torch.randn_like(x) * self.std
            return x + noise
        return x

### Base LSTM Model

In [6]:
# class LSTM(nn.Module):
#     def __init__(self, input_size, hidden_dim, output_size, num_layers):
#         super(LSTM, self).__init__()
#         # Store dimensions of the hidden state and number of layers
#         self.hidden_dim = hidden_dim
#         self.num_layers = num_layers

#         # Apply Gaussian Noise to the input for regularization (assume GaussianNoise is defined elsewhere)
#         self.noise = GaussianNoise(std=0.1)

#         # Define LSTM layers with dropout between stacked layers
#         self.lstm = nn.LSTM(
#             input_size, 
#             hidden_dim, 
#             num_layers, 
#             batch_first=True, 
#             dropout=0.3
#         )
#         self.dropout = nn.Dropout(0.3)

#         # Fully connected layer to map the hidden state output to the desired output size
#         self.fc1 = nn.Linear(hidden_dim, 64)
#         self.fc2 = nn.Linear(64, 128)
#         self.fc3 = nn.Linear(128, 64)
#         self.out = nn.Linear(64, output_size)
#         self.relu = nn.ReLU()
#         self.dropout = nn.Dropout(0.5)

#     def forward(self, x):
#         x = x.unsqueeze(1)  # Add a singleton dimension for batch_first=True
#         x = self.noise(x)  # Apply Gaussian noise

#         # Initialize hidden state h0 and cell state c0 with zeros
#         h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
#         c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)

#         # Pass the input and the initial states through the LSTM layers
#         out, _ = self.lstm(x, (h0, c0))
#         out = self.dropout(out)

#         # Pass the output of the last LSTM layer to the fully connected layer
#         out = self.relu(self.fc1(out[:, -1, :]))
#         out = self.relu(self.fc2(out))
#         out = self.dropout(out)
#         out = self.relu(self.fc3(out))
#         out = self.dropout(out)
#         out = self.out(out)
        
#         return out.squeeze()  # Squeeze the output to remove singleton dimensions

In [7]:
# import torch
# import torch.nn as nn
# from ncps.torch import LTC  # Liquid Time-Constant layer from Torch-NCPS
# from ncps.wirings import AutoNCP
# import snntorch as snn  # SNN library
# from snntorch import surrogate  # Surrogate gradient for spike training

# # Spiking Neural Network Layer with snntorch
# class SpikingLayer(nn.Module):
#     def __init__(self, input_size, output_size, beta=0.9):
#         super(SpikingLayer, self).__init__()
#         self.fc = nn.Linear(input_size, input_size*2)
#         self.fc2 = nn.Linear(input_size*2, input_size*4)
#         self.fc3 = nn.Linear(input_size*4, input_size*2)
#         self.fc4 = nn.Linear(input_size*2, output_size)
#         self.activation = nn.GELU()
#         self.encoder = nn.Sequential(self.fc,self.activation,self.fc2,self.activation,self.fc3,self.activation,self.fc4)

#     def forward(self, x):
#         # mem = self.lif.init_leaky()  # Initialize membrane potential
#         # spike, mem = self.lif(self.fc(x), mem)  # Generate spikes
#         return self.encoder(x)

# # Combined Liquid-SNN Model
# class LiquidSNN(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size, time_steps):
#         super(LiquidSNN, self).__init__()
#         # Liquid Layer using Torch-NCPS
#         wiring = AutoNCP(79, hidden_size)  # 16 units, 1 motor neuron
#         self.liquid_layer = LTC(input_size, wiring, batch_first=True)
#         self.time_steps = time_steps

#         # Spiking Layer using snntorch
#         self.spiking_layer = SpikingLayer(hidden_size, output_size)

#     def forward(self, x):
#         # Prepare input for the liquid layer (batch-first for Torch-NCPS)
#         batch_size = x.size(0)
#         x = x.unsqueeze(1).repeat(1, self.time_steps, 1)  # Repeat for time steps
        
#         # Pass through the liquid layer
#         h,state = self.liquid_layer(x)
#         # print(state.shape,h.shape)
#         # Aggregate time-dependent outputs
#         h_avg = h.mean(dim=1)  # Average across the time dimension
#         # print(h_avg.shape)/
#         # Pass through spiking layer
#         spikes = self.spiking_layer(h_avg)
#         return spikes

# # Example Usage
# if __name__ == "__main__":
#     batch_size = 4096
#     input_size = 79
#     hidden_size = 32
#     output_size = 1
#     time_steps = 1

#     # Initialize model
#     model = LiquidSNN(input_size, hidden_size, output_size, time_steps)
#     x = torch.rand(batch_size, input_size)
    
#     # Forward pass
#     output = model(x)
#     print(output.shape)


In [8]:
# class CustomTransformerNN(nn.Module):
#     def __init__(self, num_features=79, embed_dim=8, nhead=4, num_layers=2):
#         super(CustomTransformerNN, self).__init__()

#         # Simple ANN for features
#         self.feature_embeddings =  nn.Linear(1, embed_dim)

#         # Simple ANN for time_id and date_id
#         self.time_embedding = nn.Sequential(
#             nn.Linear(3, embed_dim),
#             nn.BatchNorm1d(embed_dim)
#         )
#         self.date_embedding = nn.Sequential(
#             nn.Linear(3, embed_dim),
#             nn.BatchNorm1d(embed_dim)
#         )

#         # Transformer Encoder
#         self.transformer = nn.TransformerEncoder(
#             nn.TransformerEncoderLayer(d_model=embed_dim, nhead=nhead),
#             num_layers=num_layers
#         )

#         # CNN-based autoencoder structure with final output layer
#         self.cnn_encoder = nn.Sequential(
#             nn.Conv1d(79, 64, kernel_size=3, padding=1),
#             nn.BatchNorm1d(64),
#             nn.Tanh(),
#             nn.Conv1d(64, 32, kernel_size=3, padding=1),
#             nn.BatchNorm1d(32),
#             nn.Tanh()
#         )

#         self.cnn_decoder = nn.Sequential(
#             nn.Conv1d(32, 64, kernel_size=3, padding=1),
#             nn.BatchNorm1d(64),
#             nn.Tanh(),
#             nn.Conv1d(64, 79, kernel_size=3, padding=1),
#             nn.BatchNorm1d(79),
#             nn.Tanh()
#         )

#         self.output_layer = nn.Sequential(
#             nn.Flatten(),
#             nn.Linear(79 * embed_dim, 32),
#             nn.Tanh(),
#             nn.Linear(32,1)
#         )
#         self.noise = GaussianNoise()

#     def forward(self, features, time_id, date_id):
#         # Apply linear layers to features
#         embedded_features = self.feature_embeddings(features.unsqueeze(dim=2))
#         embedded_features = self.noise(embedded_features)
#         # Apply linear layers to time_id and date_id
#         time_encoding = self.time_embedding(time_id)  # Shape: (Batch, 8)
#         date_encoding = self.date_embedding(date_id)  # Shape: (Batch, 8)

#         # Add positional encodings to the features
#         positional_encoding = time_encoding + date_encoding  # Shape: (Batch, 8)
#         positional_encoding = positional_encoding.unsqueeze(1).expand_as(embedded_features)  # (Batch, 79, 8)
#         transformer_input = embedded_features + positional_encoding

#         # Transformer Encoder
#         # transformer_output = self.transformer(transformer_input)  # Shape: (Batch, 79, 8).

#         # Reshape for CNN
#         cnn_input = transformer_input # Shape: (Batch, 8, 79)
#         encoded = self.cnn_encoder(cnn_input)  # Shape: (Batch, 32, 79)
#         decoded = self.cnn_decoder(encoded)  # Shape: (Batch, 8, 79)

#         # Flatten and pass through final output layer
#         flattened_output = decoded.permute(0, 2, 1).contiguous().view(decoded.size(0), -1)  # Shape: (Batch, 79 * 8)
#         output = self.output_layer(flattened_output)  # Shape: (Batch, 1)

#         return output

In [9]:
class AutoEncoder(nn.Module):
    def __init__(self, input_dim=79, encoded_dim=32):
        super(AutoEncoder, self).__init__()
        
        # Encoder: Maps input_dim to encoded_dim
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.BatchNorm1d(64),
            nn.SiLU(),  # Swish activation
            nn.Dropout(0.2),
            nn.Linear(64, encoded_dim),
            nn.BatchNorm1d(encoded_dim),
            nn.SiLU()  # Swish activation
        )

        # Decoder: Maps encoded_dim back to input_dim
        self.decoder = nn.Sequential(
            nn.Linear(encoded_dim, 64),
            nn.BatchNorm1d(64),
            nn.SiLU(),  # Swish activation
            nn.Dropout(0.2),
            nn.Linear(64, input_dim) # Use Sigmoid to map output in the range [0, 1], modify if not required
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

class MLP(nn.Module):
    def __init__(self, input_dim=79, encoded_dim=32, output_dim=1):
        super(MLP, self).__init__()
        self.noise =  GaussianNoise(std=0.4)
        # MLP: Takes concatenated input_dim + encoded_dim and outputs a single value
        self.mlp = nn.Sequential(
            nn.Linear( input_dim, 128),
            nn.BatchNorm1d(128),
            nn.Tanh(),  # Swish activation
            nn.Dropout(0.3),
            nn.Linear( 128, 192),
            nn.BatchNorm1d(192),
            nn.Tanh(),  # Swish activation
            nn.Dropout(0.3),
            nn.Linear(192, 16),
            # nn.BatchNorm1d(64),
            nn.Tanh(),
            nn.Dropout(0.3),
            nn.Linear(16, 1)  # Swish activation
            # nn.Dropout(0.2),
            
        )

    def forward(self, x,training = True):
        if training:
            x = self.noise(x)
        return self.mlp(x)*5

In [10]:
# input_size = 79
# hidden_size = 32
# output_size = 1
# time_steps = 1
# model = LiquidSNN(input_size, hidden_size, output_size, time_steps)
# model.load_state_dict(torch.load('/kaggle/input/snn_liquid/pytorch/default/1/torchlstm.pth',weights_only=True))


## Training Data Normalization
Found this on the forum of the competition, seemed to be extremely helpful for LSTM models.
This is a collection of the means and standard deviations for all of the feature columns in the data and placed in a dictionnary for fast pulls.
This makes the model better (increase R^2), but it also increases the overfitting, which is problematic for use on non-training data. However, like mentionned multiple times, overfitting is to be expected with LSTMs.

In [11]:
means = {'feature_00': 0.640198826789856, 'feature_01': 0.03755598142743111, 'feature_02': 0.6368075609207153, 'feature_03': 0.6365063786506653, 'feature_04': 0.013741530478000641, 'feature_05': -0.02173694409430027, 'feature_06': -0.006415014620870352, 'feature_07': -0.010971736162900925, 'feature_08': -0.04653771221637726, 'feature_09': 32.596106194690265, 'feature_10': 4.95929203539823, 'feature_11': 167.6541592920354, 'feature_12': -0.13415881991386414, 'feature_13': -0.07573335617780685, 'feature_14': -0.12015637010335922, 'feature_15': -0.7470195889472961, 'feature_16': -0.6257441639900208, 'feature_17': -0.7294047474861145, 'feature_18': -0.042215555906295776, 'feature_19': -0.08798160403966904, 'feature_20': -0.15741558372974396, 'feature_21': 0.10528526455163956, 'feature_22': 0.018054703250527382, 'feature_23': 0.03165541961789131, 'feature_24': 2.733017921447754, 'feature_25': 0.39958420395851135, 'feature_26': -0.11045943945646286, 'feature_27': -0.5332594513893127, 'feature_28': -0.4522790312767029, 'feature_29': -0.5739678144454956, 'feature_30': -0.7905704975128174, 'feature_31': 0.10600688308477402, 'feature_32': 0.40044134855270386, 'feature_33': -0.021725023165345192, 'feature_34': 0.4226262867450714, 'feature_35': 0.42143046855926514, 'feature_36': -0.00023802756913937628, 'feature_37': 0.027961043640971184, 'feature_38': 0.010258913040161133, 'feature_39': 0.005768273025751114, 'feature_40': 0.017485467717051506, 'feature_41': 0.038347117602825165, 'feature_42': -0.06123563274741173, 'feature_43': -0.11644423753023148, 'feature_44': -0.12342483550310135, 'feature_45': -0.028769943863153458, 'feature_46': -0.015200662426650524, 'feature_47': 0.015717582777142525, 'feature_48': -0.0033910537604242563, 'feature_49': -0.0052393232472240925, 'feature_50': -0.2285808026790619, 'feature_51': -0.3548349440097809, 'feature_52': -0.358092725276947, 'feature_53': 0.2607136368751526, 'feature_54': 0.18796788156032562, 'feature_55': 0.3154229521751404, 'feature_56': -0.1471923440694809, 'feature_57': 0.15730056166648865, 'feature_58': -0.021774644032120705, 'feature_59': -0.0037768862675875425, 'feature_60': -0.010220836848020554, 'feature_61': -0.03178725391626358, 'feature_62': -0.3769100308418274, 'feature_63': -0.3229374587535858, 'feature_64': -0.3718394339084625, 'feature_65': -0.10233989357948303, 'feature_66': -0.13688170909881592, 'feature_67': -0.14402112364768982, 'feature_68': -0.06875362992286682, 'feature_69': -0.11862917989492416, 'feature_70': -0.11789549142122269, 'feature_71': -0.06013699993491173, 'feature_72': -0.10766122490167618, 'feature_73': -0.09921672940254211, 'feature_74': -0.10233042389154434, 'feature_75': -0.05991339311003685, 'feature_76': -0.06349952518939972, 'feature_77': -0.07424316555261612, 'feature_78': -0.07759837061166763}
stds = {'feature_00': 1.027751088142395, 'feature_01': 1.0967519283294678, 'feature_02': 1.0156300067901611, 'feature_03': 1.0170334577560425, 'feature_04': 1.0726385116577148, 'feature_05': 0.9639211297035217, 'feature_06': 1.0963259935379028, 'feature_07': 1.0789952278137207, 'feature_08': 0.7962697148323059, 'feature_09': 23.72976726545254, 'feature_10': 3.1867162933797224, 'feature_11': 163.44513161352285, 'feature_12': 0.6700984835624695, 'feature_13': 0.5805172920227051, 'feature_14': 0.664044201374054, 'feature_15': 0.37517768144607544, 'feature_16': 0.3393096327781677, 'feature_17': 0.3603287935256958, 'feature_18': 0.9911752939224243, 'feature_19': 1.0550744533538818, 'feature_20': 0.6643751263618469, 'feature_21': 0.38239365816116333, 'feature_22': 0.950261116027832, 'feature_23': 0.8119344711303711, 'feature_24': 1.4362775087356567, 'feature_25': 1.0947270393371582, 'feature_26': 1.077124834060669, 'feature_27': 1.0645726919174194, 'feature_28': 1.0676648616790771, 'feature_29': 0.2640742361545563, 'feature_30': 0.19689509272575378, 'feature_31': 0.3815343976020813, 'feature_32': 1.2996565103530884, 'feature_33': 0.9989405870437622, 'feature_34': 1.3409572839736938, 'feature_35': 1.3365675210952759, 'feature_36': 0.8695492148399353, 'feature_37': 0.7334080934524536, 'feature_38': 0.698810338973999, 'feature_39': 0.7965824604034424, 'feature_40': 0.518515944480896, 'feature_41': 0.6384949088096619, 'feature_42': 0.8168442249298096, 'feature_43': 0.5228385925292969, 'feature_44': 0.6521403193473816, 'feature_45': 0.8666537404060364, 'feature_46': 0.9039222002029419, 'feature_47': 3.2711963653564453, 'feature_48': 0.6570901274681091, 'feature_49': 0.7083076238632202, 'feature_50': 1.0132617950439453, 'feature_51': 0.6081287860870361, 'feature_52': 0.9250587224960327, 'feature_53': 1.0421689748764038, 'feature_54': 0.5859629511833191, 'feature_55': 0.9191848039627075, 'feature_56': 0.9549097418785095, 'feature_57': 1.0204777717590332, 'feature_58': 0.8327276110649109, 'feature_59': 0.8309783339500427, 'feature_60': 0.8389413356781006, 'feature_61': 1.192766547203064, 'feature_62': 1.388945460319519, 'feature_63': 0.09957146644592285, 'feature_64': 0.3396177291870117, 'feature_65': 1.01683509349823, 'feature_66': 1.0824761390686035, 'feature_67': 0.642227828502655, 'feature_68': 0.5312599539756775, 'feature_69': 0.6208390593528748, 'feature_70': 0.6724499464035034, 'feature_71': 0.5356909036636353, 'feature_72': 0.6534596681594849, 'feature_73': 1.0855497121810913, 'feature_74': 1.0880277156829834, 'feature_75': 1.2321789264678955, 'feature_76': 1.2345560789108276, 'feature_77': 1.0921478271484375, 'feature_78': 1.0924347639083862}


In [12]:
m_arr = np.array(list(means.values()))
stds_arr = np.array(list(stds.values()))

In [13]:
means = {'feature_00': 0.640198826789856, 'feature_01': 0.03755598142743111, 'feature_02': 0.6368075609207153, 'feature_03': 0.6365063786506653, 'feature_04': 0.013741530478000641, 'feature_05': -0.02173694409430027, 'feature_06': -0.006415014620870352, 'feature_07': -0.010971736162900925, 'feature_08': -0.04653771221637726, 'feature_09': 32.596106194690265, 'feature_10': 4.95929203539823, 'feature_11': 167.6541592920354, 'feature_12': -0.13415881991386414, 'feature_13': -0.07573335617780685, 'feature_14': -0.12015637010335922, 'feature_15': -0.7470195889472961, 'feature_16': -0.6257441639900208, 'feature_17': -0.7294047474861145, 'feature_18': -0.042215555906295776, 'feature_19': -0.08798160403966904, 'feature_20': -0.15741558372974396, 'feature_21': 0.10528526455163956, 'feature_22': 0.018054703250527382, 'feature_23': 0.03165541961789131, 'feature_24': 2.733017921447754, 'feature_25': 0.39958420395851135, 'feature_26': -0.11045943945646286, 'feature_27': -0.5332594513893127, 'feature_28': -0.4522790312767029, 'feature_29': -0.5739678144454956, 'feature_30': -0.7905704975128174, 'feature_31': 0.10600688308477402, 'feature_32': 0.40044134855270386, 'feature_33': -0.021725023165345192, 'feature_34': 0.4226262867450714, 'feature_35': 0.42143046855926514, 'feature_36': -0.00023802756913937628, 'feature_37': 0.027961043640971184, 'feature_38': 0.010258913040161133, 'feature_39': 0.005768273025751114, 'feature_40': 0.017485467717051506, 'feature_41': 0.038347117602825165, 'feature_42': -0.06123563274741173, 'feature_43': -0.11644423753023148, 'feature_44': -0.12342483550310135, 'feature_45': -0.028769943863153458, 'feature_46': -0.015200662426650524, 'feature_47': 0.015717582777142525, 'feature_48': -0.0033910537604242563, 'feature_49': -0.0052393232472240925, 'feature_50': -0.2285808026790619, 'feature_51': -0.3548349440097809, 'feature_52': -0.358092725276947, 'feature_53': 0.2607136368751526, 'feature_54': 0.18796788156032562, 'feature_55': 0.3154229521751404, 'feature_56': -0.1471923440694809, 'feature_57': 0.15730056166648865, 'feature_58': -0.021774644032120705, 'feature_59': -0.0037768862675875425, 'feature_60': -0.010220836848020554, 'feature_61': -0.03178725391626358, 'feature_62': -0.3769100308418274, 'feature_63': -0.3229374587535858, 'feature_64': -0.3718394339084625, 'feature_65': -0.10233989357948303, 'feature_66': -0.13688170909881592, 'feature_67': -0.14402112364768982, 'feature_68': -0.06875362992286682, 'feature_69': -0.11862917989492416, 'feature_70': -0.11789549142122269, 'feature_71': -0.06013699993491173, 'feature_72': -0.10766122490167618, 'feature_73': -0.09921672940254211, 'feature_74': -0.10233042389154434, 'feature_75': -0.05991339311003685, 'feature_76': -0.06349952518939972, 'feature_77': -0.07424316555261612, 'feature_78': -0.07759837061166763}
stds = {'feature_00': 1.027751088142395, 'feature_01': 1.0967519283294678, 'feature_02': 1.0156300067901611, 'feature_03': 1.0170334577560425, 'feature_04': 1.0726385116577148, 'feature_05': 0.9639211297035217, 'feature_06': 1.0963259935379028, 'feature_07': 1.0789952278137207, 'feature_08': 0.7962697148323059, 'feature_09': 23.72976726545254, 'feature_10': 3.1867162933797224, 'feature_11': 163.44513161352285, 'feature_12': 0.6700984835624695, 'feature_13': 0.5805172920227051, 'feature_14': 0.664044201374054, 'feature_15': 0.37517768144607544, 'feature_16': 0.3393096327781677, 'feature_17': 0.3603287935256958, 'feature_18': 0.9911752939224243, 'feature_19': 1.0550744533538818, 'feature_20': 0.6643751263618469, 'feature_21': 0.38239365816116333, 'feature_22': 0.950261116027832, 'feature_23': 0.8119344711303711, 'feature_24': 1.4362775087356567, 'feature_25': 1.0947270393371582, 'feature_26': 1.077124834060669, 'feature_27': 1.0645726919174194, 'feature_28': 1.0676648616790771, 'feature_29': 0.2640742361545563, 'feature_30': 0.19689509272575378, 'feature_31': 0.3815343976020813, 'feature_32': 1.2996565103530884, 'feature_33': 0.9989405870437622, 'feature_34': 1.3409572839736938, 'feature_35': 1.3365675210952759, 'feature_36': 0.8695492148399353, 'feature_37': 0.7334080934524536, 'feature_38': 0.698810338973999, 'feature_39': 0.7965824604034424, 'feature_40': 0.518515944480896, 'feature_41': 0.6384949088096619, 'feature_42': 0.8168442249298096, 'feature_43': 0.5228385925292969, 'feature_44': 0.6521403193473816, 'feature_45': 0.8666537404060364, 'feature_46': 0.9039222002029419, 'feature_47': 3.2711963653564453, 'feature_48': 0.6570901274681091, 'feature_49': 0.7083076238632202, 'feature_50': 1.0132617950439453, 'feature_51': 0.6081287860870361, 'feature_52': 0.9250587224960327, 'feature_53': 1.0421689748764038, 'feature_54': 0.5859629511833191, 'feature_55': 0.9191848039627075, 'feature_56': 0.9549097418785095, 'feature_57': 1.0204777717590332, 'feature_58': 0.8327276110649109, 'feature_59': 0.8309783339500427, 'feature_60': 0.8389413356781006, 'feature_61': 1.192766547203064, 'feature_62': 1.388945460319519, 'feature_63': 0.09957146644592285, 'feature_64': 0.3396177291870117, 'feature_65': 1.01683509349823, 'feature_66': 1.0824761390686035, 'feature_67': 0.642227828502655, 'feature_68': 0.5312599539756775, 'feature_69': 0.6208390593528748, 'feature_70': 0.6724499464035034, 'feature_71': 0.5356909036636353, 'feature_72': 0.6534596681594849, 'feature_73': 1.0855497121810913, 'feature_74': 1.0880277156829834, 'feature_75': 1.2321789264678955, 'feature_76': 1.2345560789108276, 'feature_77': 1.0921478271484375, 'feature_78': 1.0924347639083862}

def normalize_dataframe(df: pl.DataFrame, means: dict, stds: dict) -> pl.DataFrame:
    # We normalize the polars dataframe using the provided means and standard deviations
    normalize_exprs = []

    for col in df.columns:
        if col in means and col in stds: #only normalize columns present in the means and std
            if stds[col] != 0: #avoid division by 0
                #Normalize the column and alias it with the same name
                normalize_exprs.append(
                    ((pl.col(col) - means[col]) / stds[col]).alias(col)
                )
            else:
                normalize_exprs.append(pl.col(col) - means[col]).alias(col)

    normalized_df = df.select(normalize_exprs) #Dataframe with normalized expressions
    return normalized_df



## Preparing the Training Data
Date to start the training data from, data collection gotten from https://www.kaggle.com/code/chumajin/janestreet-updated-simulator-for-time-series-api

In [14]:
train_range = [1455,1672]
val_range = [1673,1693]
test_range = [1694,1698]

In [15]:
def convert_to_vector(date_ids, time_ids):
    date_ids = np.array(date_ids).reshape(-1, 1)
    time_ids = np.array(time_ids).reshape(-1, 1)
    # print(date_ids)/
    # print(data_ids.shape)
    date_vectors = np.hstack((date_ids / 30, date_ids / 12, date_ids / 365))
    # print(date_vectors)
    # print("hrerer")
    time_vectors = np.hstack((time_ids / 16, time_ids / 60, time_ids / 360))
    
    return date_vectors, time_vectors

In [16]:
def get_features(train):
    feature_names = [f"feature_{i:02d}" for i in range(79)]
    train_features = train.select(feature_names)
    train_features = train_features.fill_null(strategy='forward').fill_null(0)
    
    # date_ids,time_ids = convert_to_vector(date_ids, time_ids)
    # train_features = normalize_dataframe(train_features,means,stds)
    X = train_features.to_numpy()
    del train_features
    y = train.select('responder_6').to_numpy().reshape(-1)
    weights = train.select('weight').to_numpy().reshape(-1)

    return X,y,weights
    

In [17]:
# date_ids_train.shape,weights_train.shape 

## Data Train Validation Test Split
Use 80% of the data for training 10% for validation and early stopping, 10% for final testing.

In [18]:
import random
import gc

alltraindata = pl.scan_parquet(f"{jane_street_real_time_market_data_forecasting_path}/train.parquet")

# Get unique date_ids greater than 1400 and shuffle them
unique_date_ids = alltraindata.select("date_id").unique().collect().to_series().to_list()
unique_date_ids = [date_id for date_id in unique_date_ids if date_id > 1400]

# Separate dates 1690 to 1698 for the test set
test_date_ids_specific = [date_id for date_id in unique_date_ids if 1690 <= date_id <= 1698]
remaining_date_ids = [date_id for date_id in unique_date_ids if date_id not in test_date_ids_specific]
random.shuffle(remaining_date_ids)

# Split remaining date_ids into train and val subsets
train_size = int(len(remaining_date_ids) * 0.7)  # 70% for training
val_size = len(remaining_date_ids) - train_size  # Remaining for validation

train_date_ids = remaining_date_ids[:train_size]
val_date_ids = remaining_date_ids[train_size:]
test_date_ids = test_date_ids_specific

# Filter data based on date_id splits
train = alltraindata.filter(pl.col("date_id").is_in(train_date_ids)).collect()
X_train, y_train, weights_train  = get_features(train)
del train
gc.collect()
val = alltraindata.filter(pl.col("date_id").is_in(val_date_ids)).collect()
X_val, y_val, weights_val = get_features(val)
del val
gc.collect()
test = alltraindata.filter(pl.col("date_id").is_in(test_date_ids)).collect()
X_test, y_test, weights_test = get_features(test)
gc.collect()

0

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [20]:
train_X = torch.tensor(X_train,dtype=torch.float16).to(device)
train_y = torch.tensor(y_train, dtype=torch.float32).to(device)
train_weights = torch.tensor(weights_train,dtype=torch.float32).to(device)
# train_time_ids = torch.tensor(date_ids_train,dtype=torch.float32).to(device)
# train_date_ids = torch.tensor(time_ids_train,dtype=torch.float32).to(device)

val_X = torch.tensor(X_val,dtype=torch.float32).to(device)
val_y = torch.tensor(y_val, dtype=torch.float32).to(device)
val_weights = torch.tensor(weights_val,dtype=torch.float32).to(device)
# val_time_ids = torch.tensor(date_ids_val,dtype=torch.float32).to(device)
# val_date_ids = torch.tensor(time_ids_val,dtype=torch.float32).to(device)

test_X = torch.tensor(X_test,dtype=torch.float32).to(device)
test_y = torch.tensor(y_test, dtype=torch.float32).to(device)
test_weights = torch.tensor(weights_test,dtype=torch.float32).to(device)
# test_time_ids = torch.tensor(date_ids_test,dtype=torch.float32).to(device)
# test_date_ids = torch.tensor(time_ids_test,dtype=torch.float32).to(device)


train_X.shape

torch.Size([7436176, 79])

In [21]:
# train_time_ids.shape,test_date_ids.shape

m_arr = torch.tensor(m_arr,dtype=torch.float32).to(device)
stds= torch.tensor(stds_arr,dtype=torch.float32).to(device)

In [22]:
class CustomDataset(Dataset):
    def __init__(self,X , y, weights):
        self.X = X
        self.y = y
        self.weights = weights
        
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (self.X[idx]-m_arr)/stds, self.y[idx],self.weights[idx]

In [23]:
train_dataset = CustomDataset(train_X, train_y,train_weights)
val_dataset = CustomDataset(val_X, val_y, val_weights)
test_dataset = CustomDataset(test_X, test_y, test_weights)

In [24]:
batch_size = 4096
train_loader = DataLoader(train_dataset, batch_size=batch_size,shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size,shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size,shuffle=False)

In [25]:
import os
run_type = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', 'Local')


#### Model

In [26]:
# encoder_path = "/kaggle/input/encoder/pytorch/default/1/feature_learner.pth"
# encoder = AutoEncoder()
# # model = nn.DataParallel(model)
# encoder.to(device)
# encoder.load_state_dict(torch.load(encoder_path, weights_only=True))

In [27]:
if evaluate:
    print("Loading pretrained weights")
    model.noise.training = False
    model.load_state_dict(torch.load(model_path, weights_only=True))
    model.eval()

In [28]:
# weighted r2 gotten from https://www.kaggle.com/code/chumajin/janestreet-updated-simulator-for-time-series-api
def r2_score(y_true, y_pred, weights):
    """
    Calculate the sample weighted zero-mean R-squared score.

    Parameters:
    y_true (numpy.ndarray): Ground-truth values for responder_6.
    y_pred (numpy.ndarray): Predicted values for responder_6.
    weights (numpy.ndarray): Sample weight vector.

    Returns:
    float: The weighted zero-mean R-squared score.
    """
    numerator = np.sum(weights * (y_true - y_pred)**2)
    denominator = np.sum(weights * y_true**2)

    r2_score = 1 - numerator / denominator
    return r2_score

## Training Loop

In [29]:
from tqdm import tqdm


In [30]:
from sklearn.metrics import mean_squared_error
def train_model(model, encoder,loader, optimizer, loss_function, device):
    model.train() #set model to training mode
    total_loss = 0
    all_probs = []
    all_targets = []
    all_weights = []
    # model.noise.training = True
    #Iterate over batches 
    progress_bar = tqdm(loader, desc="Training Progress", leave=True,position =0)
    for X_batch, y_batch, weights_batch in progress_bar:
        #Move data to specified device (CPU or GPU)
        # X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        # weights_batch = weights_batch.to(device)
        #Reset Gradient to 0
        optimizer.zero_grad()
        # print(batch_time.shape,batch_date.shape)
        # encoded,_ = encoder(X_batch)
        # concat = torch.cat((X_batch, encoded), dim=1)
        outputs = model(X_batch)
        outputs = outputs.squeeze(dim=1)
        y_batch = y_batch
        loss_per_sample = loss_function(outputs, y_batch)
        weighted_loss = loss_per_sample*weights_batch
        #Compute average loss across the batch
        loss = weighted_loss.mean()
        # print(f"Batch Loss: {loss.item():.4f}")
        progress_bar.set_postfix({"Batch Loss": f"{loss.item():.4f}"})

        loss.backward()

        #Update model parameters
        optimizer.step()

        total_loss += loss.item()

        all_probs.append(outputs.detach().cpu())
        all_targets.append(y_batch.cpu())
        all_weights.append(weights_batch.cpu())

    all_probs = torch.cat(all_probs).numpy()
    all_targets = torch.cat(all_targets).numpy()
    all_weights = torch.cat(all_weights).numpy()
    mse = mean_squared_error(all_targets, all_probs, sample_weight=all_weights)
    r2 = r2_score(all_targets, all_probs, all_weights)

    avg_loss = total_loss / len(loader)
    return avg_loss, mse, r2

In [31]:
def evaluate_model(model, encoder,loader,generate_preds = False):
    model.eval()
    all_probs = []
    all_targets = []
    all_weights = []
    # model.noise.training = False
    with torch.no_grad():
        for X_batch, y_batch, weights_batch in tqdm(loader, desc="Validating Progress", leave=True,position=0):
            y_batch = y_batch
            optimizer.zero_grad()
            # print(batch_time.shape,batch_date.shape)
            # encoded,_ = encoder(X_batch)
            # concat = torch.cat((X_batch, encoded), dim=1)
            outputs = model(X_batch,False)

            all_probs.append(outputs.squeeze(dim=1).cpu())
            all_targets.append(y_batch.cpu())
            all_weights.append(weights_batch.cpu())

    all_probs = torch.cat(all_probs).numpy()
    all_targets = torch.cat(all_targets).numpy()
    all_weights = torch.cat(all_weights).numpy()
    
    mse = mean_squared_error(all_targets, all_probs, sample_weight=all_weights)
    r2 = r2_score(all_targets, all_probs, all_weights)
    print(f"Total Loss val: {mse:.4f} r2 {r2}")
    if generate_preds:
        return all_targets,all_probs,mse,r2
    return mse, r2

In [32]:
# from sklearn.metrics import mean_squared_error
# def train_model(model, loader, optimizer, loss_function, device):
#     model.train() #set model to training mode
#     total_loss = 0
#     all_probs = []
#     all_targets = []
#     all_weights = []
#     model.noise.training = True
#     #Iterate over batches 
#     progress_bar = tqdm(loader, desc="Training Progress", leave=True,position =0)
#     for X_batch, y_batch, batch_time,batch_date,weights_batch in progress_bar:
#         #Move data to specified device (CPU or GPU)
#         # X_batch, y_batch = X_batch.to(device), y_batch.to(device)
#         # weights_batch = weights_batch.to(device)
#         #Reset Gradient to 0
#         optimizer.zero_grad()
#         # print(batch_time.shape,batch_date.shape)
#         outputs = model(X_batch,batch_time,batch_date)
#         outputs = outputs.squeeze(dim=1)
#         loss_per_sample = loss_function(outputs, y_batch)
#         weighted_loss = loss_per_sample
#         #Compute average loss across the batch
#         loss = weighted_loss
#         # print(f"Batch Loss: {loss.item():.4f}")
#         progress_bar.set_postfix({"Batch Loss": f"{loss.item():.4f}"})

#         loss.backward()

#         #Update model parameters
#         optimizer.step()

#         total_loss += loss.item()

#         all_probs.append(outputs.detach().cpu())
#         all_targets.append(y_batch.cpu())
#         all_weights.append(weights_batch.cpu())

#     all_probs = torch.cat(all_probs).numpy()
#     all_targets = torch.cat(all_targets).numpy()
#     all_weights = torch.cat(all_weights).numpy()
#     mse = mean_squared_error(all_targets, all_probs, sample_weight=all_weights)
#     r2 = r2_score(all_targets, all_probs, all_weights)

#     avg_loss = total_loss / len(loader)
#     return avg_loss, mse, r2

In [33]:
for X_batch, y_batch,weights_batch in train_loader:
    print(weights_batch.shape)
    break

torch.Size([4096])


In [34]:
# def evaluate_model(model, loader,generate_preds = False):
#     model.eval()
#     all_probs = []
#     all_targets = []
#     all_weights = []
#     model.noise.training = False
#     with torch.no_grad():
#         for X_batch, y_batch, batch_time,batch_date,weights_batch in tqdm(loader, desc="Validating Progress", leave=True,position=0):
#             outputs = model(X_batch,batch_time,batch_date)

#             all_probs.append(outputs.squeeze(dim=1).cpu())
#             all_targets.append(y_batch.cpu())
#             all_weights.append(weights_batch.cpu())

#     all_probs = torch.cat(all_probs).numpy()
#     all_targets = torch.cat(all_targets).numpy()
#     all_weights = torch.cat(all_weights).numpy()
    
#     mse = mean_squared_error(all_targets, all_probs, sample_weight=all_weights)
#     r2 = r2_score(all_targets, all_probs, all_weights)
#     print(f"Total Loss val: {mse:.4f} r2 {r2}")
#     if generate_preds:
#         return all_targets,all_probs,mse,r2
#     return mse, r2

In [35]:
from sklearn.metrics import mean_squared_error
def train_ae(model, loader, optimizer, loss_function, device):
    model.train() #set model to training mode
    #Iterate over batches 
    total_loss = 0
    all_probs = []
    all_targets = []
    all_weights = []
    progress_bar = tqdm(loader, desc="Training Progress", leave=True,position =0)
    for X_batch, y_batch, batch_time,batch_date,weights_batch in progress_bar:
        #Move data to specified device (CPU or GPU)
        # X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        # weights_batch = weights_batch.to(device)
        #Reset Gradient to 0
        optimizer.zero_grad()
        # print(batch_time.shape,batch_date.shape)
        _,outputs = model(X_batch)
        outputs = outputs
        loss_per_sample = loss_function(outputs, X_batch)
        weighted_loss = loss_per_sample
        #Compute average loss across the batch
        loss = weighted_loss
        # print(f"Batch Loss: {loss.item():.4f}")
        progress_bar.set_postfix({"Batch Loss": f"{loss.item():.4f}"})

        loss.backward()

        #Update model parameters
        optimizer.step()

        total_loss += loss.item()

        all_probs.append(outputs.detach().cpu())
        all_targets.append(X_batch.cpu())
        all_weights.append(weights_batch.cpu())

    all_probs = torch.cat(all_probs).numpy()
    all_targets = torch.cat(all_targets).numpy()
    all_weights = torch.cat(all_weights).numpy()
    mse = mean_squared_error(all_targets, all_probs)
    r2 = 0

    avg_loss = total_loss / len(loader)
    return avg_loss, mse, r2

In [36]:
from sklearn.metrics import mean_squared_error
def evaluate_ae(model, loader, optimizer, loss_function, device):
    model.train() #set model to training mode
    #Iterate over batches 
    progress_bar = tqdm(loader, desc="Validating Progress", leave=True,position =0)
    total_loss = 0
    all_probs = []
    all_targets = []
    all_weights = []
    with torch.no_grad():
        for X_batch, y_batch, batch_time,batch_date,weights_batch in progress_bar:
            #Move data to specified device (CPU or GPU)
            # X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            # weights_batch = weights_batch.to(device)
            #Reset Gradient to 0
            optimizer.zero_grad()
            # print(batch_time.shape,batch_date.shape)
            _,outputs = model(X_batch)
            outputs = outputs
            loss_per_sample = loss_function(outputs, X_batch)
            weighted_loss = loss_per_sample
            #Compute average loss across the batch
            loss = weighted_loss
            # print(f"Batch Loss: {loss.item():.4f}")
            progress_bar.set_postfix({"Batch Loss": f"{loss.item():.4f}"})
    
            total_loss += loss.item()
    
            all_probs.append(outputs.detach().cpu())
            all_targets.append(X_batch.cpu())
            all_weights.append(weights_batch.cpu())

    all_probs = torch.cat(all_probs).numpy()
    all_targets = torch.cat(all_targets).numpy()
    all_weights = torch.cat(all_weights).numpy()
    mse = mean_squared_error(all_targets, all_probs, sample_weight=all_weights)
    r2 = 0

    avg_loss = total_loss / len(loader)
    return mse, r2

In [37]:
# !pip install matplotlib

In [38]:
import matplotlib.pyplot as plt

def plot(targets,probs,range):
    pred = probs[range[0]:range[1]]
    gt = targets[range[0]:range[1]]
    plt.figure(figsize=(10, 6))
    plt.plot(gt, label='Ground Truth', color='royalblue', linewidth=2)
    plt.style.use('dark_background')  # Use a dark background style
    # Plot predictions
    plt.plot(pred, label='Predictions', color='tomato', linestyle='--', linewidth=2)
    
    # Add title and labels
    plt.title('Ground Truth vs Predictions', fontsize=16, fontweight='bold')
    plt.xlabel('Index', fontsize=14)
    plt.ylabel('Value', fontsize=14)
    
    # Add a grid
    plt.grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.7)
    
    # Add a legend
    plt.legend(fontsize=12, loc='upper right')
    
    # Customize tick parameters
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    
    # Show the plot
    plt.tight_layout()
    plt.show()

## Training Model
Use very few epochs because like mentionned lots of times, LSTM are very proned to overfitting. Also, the dataset is very large, more epochs seem to be detrimental with this type of model.

In [39]:
evaluate

False

In [40]:
mlp = MLP()
mlp = mlp.to(device)

In [41]:
# encoder.requires_grad = False

In [42]:

optimizer = optim.Adam(mlp.parameters(), lr=1e-4) #Adam optimizer
loss_function = nn.MSELoss(reduction='none')
epochs = 30
best = float('-inf')
degraded = 0
best_model = mlp
if evaluate:
    targets,probs,val_mse, val_r2 = evaluate_model(mlp,None, test_loader,generate_preds=evaluate)
else:
    for epoch in range(epochs):
        train_loss, train_mse, train_r2 = train_model(mlp,None, train_loader, optimizer, loss_function, device)
        val_mse, val_r2 = evaluate_model(mlp,None, val_loader)
    
        print(f'epoch {epoch} train loss {train_loss:.4f}, train_r2 {train_r2:.4f}, train_mse {train_mse:.4f}, val_mse {val_mse:.4f}, val_r2 {val_r2:.4f}')
        if val_r2 > best:
            best = val_r2
            best_model = copy.deepcopy(mlp)
            torch.save(best_model.state_dict(), f'torchcustomwithencoder.pth')
            degraded = 0
            targets,probs,val_mse, val_r2 = evaluate_model(mlp, None,test_loader,generate_preds=True)
            # plot(targets,probs,[245,455])
        else:
            degraded += 1
        if degraded > 5:
            break
    model = mlp


Validating Progress: 100%|██████████| 777/777 [00:52<00:00, 14.70it/s]


Total Loss val: 0.6926 r2 0.0020408034324645996
epoch 0 train loss 2.0494, train_r2 -0.3576, train_mse 0.8033, val_mse 0.6926, val_r2 0.0020


Validating Progress: 100%|██████████| 83/83 [00:05<00:00, 14.87it/s]


Total Loss val: 0.5239 r2 0.0007173418998718262


Validating Progress: 100%|██████████| 777/777 [00:51<00:00, 15.11it/s]


...

Total Loss val: 0.6882 r2 0.008404850959777832
epoch 28 train loss 1.4925, train_r2 0.0114, train_mse 0.5850, val_mse 0.6882, val_r2 0.0084


Validating Progress: 100%|██████████| 777/777 [00:50<00:00, 15.42it/s]


Total Loss val: 0.6881 r2 0.008597612380981445
epoch 29 train loss 1.4922, train_r2 0.0115, train_mse 0.5849, val_mse 0.6881, val_r2 0.0086


Validating Progress: 100%|██████████| 83/83 [00:05<00:00, 15.60it/s]

Total Loss val: 0.5227 r2 0.0030341744422912598


In [41]:
i = 0

In [42]:
# import matplotlib.pyplot as plt


# if True:
#     plot(targets,probs,[1000*i,1000*(i+1)])
#     i+=1

## Upload model

In [43]:
# Save only the model's parameters 
# if run_type == 'Interactive':
#     torch.save(model.state_dict(), 'lstm_model_weights.pth')

In [44]:
# print("hrerer")

## Load model

In [45]:
# Load the model's parameters
# if run_type == 'Interactive':
#     input_size = 79
#     hidden_size = 32
#     output_size = 1
#     time_steps = 1
#     loaded_model = LiquidSNN(input_size, hidden_size, output_size, time_steps)
#     loaded_model.load_state_dict(torch.load('/kaggle/input/snn_liquid/pytorch/default/1/torchlstm.pth',weights_only=True))

#     # loaded_model = LSTM(input_size=79, hidden_dim=hidden_dim, output_size=1, num_layers=num_layer) # Recreate the model structure
#     # loaded_model.load_state_dict(torch.load('lstm_model_weights.pth'))
#     loaded_model.eval()  # Set the model to evaluation mode

## Submission

In [46]:
# def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
#     """Make a prediction."""
#     global model
#     global device
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     # Load the model if it's not already loaded

    
#     model = LiquidSNN(input_size, hidden_size, output_size, time_steps)
#     model.load_state_dict(torch.load('/kaggle/input/snn_liquid/pytorch/default/1/torchlstm.pth',weights_only=True))
#     model = model.to(device)
#     # loaded_model = LSTM(input_size=79, hidden_dim=hidden_dim, output_size=1, num_layers=num_layer) # Recreate the model structure
#     # loaded_model.load_state_dict(torch.load('lstm_model_weights.pth'))
#     model.eval()  # Set the model to evaluation mode
#     sel_cols  = [f"feature_{i:02d}" for i in range(79)]

#     # Process the test data using Polars
#     # Select the same features used during training


#     # Ensure all required columns are present
#     missing_cols = set(sel_cols) - set(test.columns)
#     if missing_cols:
#         raise ValueError(f"Missing columns in test data: {missing_cols}")

#     # Select the features
#     test_features = test.select(sel_cols)

#     # **Apply forward fill and then fill remaining missing values with zero**
#     test_features = test_features.fill_null(strategy='forward').fill_null(0)
#     test_features = normalize_dataframe(test_features,means,stds)

#     # Convert Polars DataFrame to NumPy array
#     X_test = test_features.to_numpy()

#     # Convert to Torch tensor
#     X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)

#     # Make predictions
#     with torch.no_grad():
        
#         outputs = model(X_test_tensor)
#         # Assuming the model outputs a tensor of shape (batch_size, 1)
#         predictions = outputs.squeeze().cpu().numpy()

#     # Prepare the predictions DataFrame
#     predictions_df = pl.DataFrame({
#         'row_id': test['row_id'],
#         'responder_6': predictions
#     })

#     # The predict function must return a DataFrame
#     assert isinstance(predictions_df, (pl.DataFrame, pd.DataFrame))
#     # with columns 'row_id', 'responder_6'
#     assert predictions_df.columns == ['row_id', 'responder_6']
#     # and as many rows as the test data.
#     assert len(predictions_df) == len(test)

#     return predictions_df

In [47]:
# import os

# import torch
# import pandas as pd
# import polars as pl

# import kaggle_evaluation.jane_street_inference_server


# inference_server = \
# kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)

# if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
#     inference_server.serve()
# else:
#     inference_server.run_local_gateway(
#         (
#             '/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet',
#             '/kaggle/input/jane-street-real-time-market-data-forecasting/lags.parquet',
#         )
#     )